In [1]:
%matplotlib ipympl
import random
import math

In [2]:
class memory:
    def __init__(self, size):
        self.size = size # memory size in bytes
        self.free_mem = size
        self.memory = [ False ] * size
    
    def __str__(self):
        str1  = "Size: {} bytes\n".format(str(self.size))
        str1 += "Free: {} bytes\n".format(str(self.free_mem))
        str1 += str(list(int(i) for i in self.memory))
        return str1
                
    def writeAllocation(self, position, size):
        self.memory[position: position+size] = [True] * size
            
    def isThereSpace(self, position, size):
        if position+size > self.size:
            return 0
        return sum(self.memory[position : position+size]) == 0

    def get_number_free_chunks(self, size):
        num = 0
        i = 0
        while i < self.memory.size:
            if self.memory.isThereSpace(i,size):
                num += 1
                i += size
            else:
                i += 1
        return num
        
    def get_fragmentation(self, size_x=4):
        if size_x > self.memory.free_mem:
            return 1
        num_free_chunks_of_size_x = self.get_number_free_chunks(size_x)
        return 1-size_x*num_free_chunks_of_size_x/self.memory.free_mem

In [3]:
class random_allocator:        
    def allocate(self, size, memory):
        allocated = False
        while not allocated:
            pick_place_to_allocate = random.randint(0,memory.size-1-size)
            if memory.isThereSpace(pick_place_to_allocate, size):
                memory.writeAllocation(pick_place_to_allocate, size)
                memory.free_mem -= size
                allocated = True

In [4]:
class sequential_allocator:        
    def allocate(self, size, memory):
        memory.writeAllocation(memory.size-memory.free_mem, size)
        memory.free_mem -= size

https://github.com/mtrebi/memory-allocators (CTRL+F -> Pool allocator)

In [5]:
class pool_allocator:
    chunk_size = 0
    memory_chunk_positions = []
    memory_chunks = []
    memory_ = None
    allocated = []
    
    def __init__(self, chunk_size, memory):
        self.chunk_size = chunk_size
        self.memory_position = [0]
        if memory.size % chunk_size != 0: print("Memory size to be allocated and chunk size decided for pool allocator should be a perfect division!")
        self.memory_buddies = [chunk_size for x in range(int(memory.size / chunk_size))]
        self.memory_ = memory
    
    def __str__(self):
        str_ =  "Positions: " + str([int(x) for x in self.memory_position]) + "\n"
        str_ += "Pool:   " + str([int(x) for x in self.memory_buddies]) + "\n"
        str_ += "Allocated: " + str([(int(x), int(y)) for (x,y) in self.allocated])
        return str_
    
    def deallocate(self, pos, memory=None):
        if memory == None: memory = self.memory_
        
        found = False
        
        for (x,y) in self.allocated:
            if pos == x:
                found = True
                self.allocated.remove((x,y))
                position = x
                self.memory_positions.append(position)
                self.memory_positions.sort()
                index = self.memory_positions.index(position)
                buddyToPutBack = math.ceil(math.sqrt(y))
                
                self.memory_buddies[index:index+1] = [buddyToPutBack]
                                                          
                return 0
                
        if found == False: return -1
            
    def findSpace(self, chunks_needed):
        for i in range(len(self.memory_chunk_positions)):
            didnt_find_it = False
            print(i)
            for j in range(chunks_needed):
                if self.memory_chunk_positions[i]+(chunk_size*j) != self.memory_chunk_positions[i+j]:
                    didnt_find_it = True
                    print(str(i),str(j))
            
            if didnt_find_it == False:
                return i
        return -1
    
    def allocate(self, size, memory=None):
        if memory == None: memory = self.memory_
        
        chunks_needed = math.ceil(size / self.chunk_size)
        
        allocate_at = self.findSpace(chunks_needed)
        
        print("Allocated at " + str(int(allocate_at)))
        return (allocate_at, size)

In [6]:
memory_ = memory(16)
allocator = pool_allocator(2, memory_)

print(str(allocator) + "\n")

allocator.allocate(3)

# print("\n")
# print(str(allocator))

# allocator.allocate(3)

# print("\n")
# print(str(allocator))

# allocator.deallocate(4)

# print("\n")
# print(str(allocator))

Positions: [0]
Pool:   [2, 2, 2, 2, 2, 2, 2, 2]
Allocated: []

Allocated at -1


(-1, 3)